In [1]:
import torch
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# Import your CNN and LSTM training modules
# Ensure these imports match your project structure
from CNN_ensemble import train_model as train_cnn, load_cnn_data as load_cnn_data
from LSTM import train_model as train_lstm, load_lstm_data as load_lstm_data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

def evaluate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    accuracy = torch.sum(preds == labels).item() / len(preds)
    return accuracy * 100

def combine_models(cnn_model, lstm_model, test_loader_cnn, test_loader_lstm):
    cnn_model.eval()
    lstm_model.eval()
    combined_accuracy = 0
    with torch.no_grad():
        for (images, labels), (numerical, _) in zip(test_loader_cnn, test_loader_lstm):
            images, labels, numerical = images.to(device), labels.to(device), numerical.to(device)
            cnn_outputs = cnn_model(images)
            lstm_outputs = lstm_model(numerical)
            combined_outputs = (F.softmax(cnn_outputs, dim=1) + F.softmax(lstm_outputs, dim=1)) / 2
            combined_accuracy += evaluate_accuracy(combined_outputs, labels)
    combined_accuracy /= len(test_loader_cnn)
    return combined_accuracy

if __name__ == "__main__":
    # Train CNN and LSTM models
    cnn_model = train_cnn()
    lstm_model = train_lstm()


    # Load test data
    _, _, test_images, test_labels = load_cnn_data()
    test_numerical, test_labels = load_lstm_data()[2:]  

    # Prepare DataLoaders
    test_loader_cnn = DataLoader(TensorDataset(test_images, test_labels), batch_size=64, shuffle=False)
    test_loader_lstm = DataLoader(TensorDataset(test_numerical, test_labels), batch_size=64, shuffle=False)


    # Combine models and evaluate on test set
    combined_test_accuracy = combine_models(cnn_model, lstm_model, test_loader_cnn, test_loader_lstm)
    print(f'Combined Model Test Accuracy: {combined_test_accuracy:.2f}%')


Using cuda device
Using cuda device
Using cuda device
Epoch 1/25, Loss: 0.3711, Train Accuracy: 74.50%, Test Accuracy: 58.85%
Epoch 2/25, Loss: 0.0889, Train Accuracy: 85.91%, Test Accuracy: 67.36%
Epoch 3/25, Loss: 0.0667, Train Accuracy: 87.43%, Test Accuracy: 68.70%
Epoch 4/25, Loss: 0.0366, Train Accuracy: 90.61%, Test Accuracy: 70.07%
Epoch 5/25, Loss: 0.0388, Train Accuracy: 95.22%, Test Accuracy: 76.68%
Epoch 6/25, Loss: 0.0313, Train Accuracy: 66.09%, Test Accuracy: 62.05%
Epoch 7/25, Loss: 0.0262, Train Accuracy: 91.92%, Test Accuracy: 71.70%
Epoch 8/25, Loss: 0.0196, Train Accuracy: 99.60%, Test Accuracy: 73.75%
Epoch 9/25, Loss: 0.0240, Train Accuracy: 69.65%, Test Accuracy: 53.80%
Epoch 10/25, Loss: 0.0179, Train Accuracy: 99.78%, Test Accuracy: 77.49%
Epoch 11/25, Loss: 0.0012, Train Accuracy: 100.00%, Test Accuracy: 77.79%
Epoch 12/25, Loss: 0.0005, Train Accuracy: 100.00%, Test Accuracy: 77.70%
Epoch 13/25, Loss: 0.0003, Train Accuracy: 100.00%, Test Accuracy: 78.46%
Epo

c:\Users\chris\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/25, Loss: 0.9807, Train Acc: 57.43%, Test Acc: 50.44%
Epoch 2/25, Loss: 0.8377, Train Acc: 58.74%, Test Acc: 54.21%
Epoch 3/25, Loss: 0.7985, Train Acc: 63.97%, Test Acc: 52.44%
Epoch 4/25, Loss: 0.7508, Train Acc: 71.11%, Test Acc: 56.48%
Epoch 5/25, Loss: 0.5878, Train Acc: 77.46%, Test Acc: 58.16%
Epoch 6/25, Loss: 0.4427, Train Acc: 89.59%, Test Acc: 60.23%
Epoch 7/25, Loss: 0.3279, Train Acc: 89.21%, Test Acc: 61.79%
Epoch 8/25, Loss: 0.2107, Train Acc: 96.96%, Test Acc: 62.46%
Epoch 9/25, Loss: 0.1588, Train Acc: 97.66%, Test Acc: 62.91%
Epoch 10/25, Loss: 0.1056, Train Acc: 98.87%, Test Acc: 63.29%
Epoch 11/25, Loss: 0.0762, Train Acc: 99.03%, Test Acc: 65.80%
Epoch 12/25, Loss: 0.0633, Train Acc: 99.67%, Test Acc: 64.05%
Epoch 13/25, Loss: 0.1063, Train Acc: 92.17%, Test Acc: 61.41%
Epoch 14/25, Loss: 0.0951, Train Acc: 99.27%, Test Acc: 64.29%
Epoch 15/25, Loss: 0.0603, Train Acc: 99.70%, Test Acc: 65.14%
Epoch 16/25, Loss: 0.0545, Train Acc: 99.86%, Test Acc: 65.10%
E